In [1]:
import subprocess
import logging
import os
import numpy as np
import random
import shutil
import platform
import sys
import gzip
from typing import Union, List, Optional, Tuple

In [2]:
class DependencyError(Exception):
    pass

In [3]:
class File(object):
    '''Creates File object that encapsulates a number of methods and properites
    for file and filename handling.
    
    Attributes (class and instance attributes):
        file (class and instance): Class variable that is set once class is instantiated.
    '''
    
    file: str = ""
    
    def __init__(self,
                 file: str,
                 ext: str = "") -> None:
        '''Init doc-string for File object class.
        
        Usage example:
            file_obj = File("<file_name.txt>")
            file_obj.file -> "<file_name.txt>" # Returns filename str
        
        Args:
            file: Input file (need not exist at runtime/instantiated).
            ext: File extension of input file. 
        '''
        self.file: str = file
        if ext:
            self.ext: str = ext
        elif '.gz' in self.file:
            self.ext: str = self.file[-(7):]
        else:
            self.ext: str = self.file[-(4):]
        
    def touch(self) -> str:
        '''Creates empty file.
        
        Usage example:
            file_obj = File("<file_name.txt>")
            file_obj.touch()   # Creates empty file
        '''
        with open(self.file,'w') as tmp_file:
            pass
        return self.file
    
    def abs_path(self) -> str:
        '''Returns absolute path of file.
        
        Usage example:
            file_obj = File("<file_name.txt>")
            file_obj.abs_path()   # Returns abs path of file as str
        '''
        if os.path.exists(self.file):
            return os.path.abspath(self.file)
        else:
            self.touch()
            file_path = os.path.abspath(self.file)
            os.remove(self.file)
            return file_path
    
    def rm_ext(self,
              ext: str = "") -> str:
        '''Removes file extension from the file.
        
        Usage example:
            # Recommended
            file_obj = File("<file_name.txt>")
            file_obj.rm_ext(".txt")
            
            # or
            
            # Not recommended
            file_obj = File("<file_name.txt>")
            file_obj.rm_ext()   # Returns "file_name"
        
        Args:
            ext: File extension.
        
        Returns:
            file: File name with no extension.
        '''
        if ext:
            ext_num = len(ext)
            return self.file[:-(ext_num)]
        elif self.ext:
            ext_num = len(self.ext)
            return self.file[:-(ext_num)]
        else:
            return self.file[:-(4)]
        
    def write_txt(self,
                 txt: str = "") -> str:
        '''Writes/appends text to file.
        
        Usage example:
            file_obj = File("<file_name.txt>")
            file_obj.write_txt("<Some text to be written>")
        
        Args:
            txt: Text to be written to file.
        Returns:
            file: File with text written/appended to it.
        '''
        with open(self.file,"a") as tmp_file:
            tmp_file.write(txt)
            tmp_file.close()
        return self.file
    
    def file_parts(self,
                  ext: str = "") -> Tuple[str,str,str]:
        '''Splits a file and its path into its constituent 
        parts:
            * file path
            * filename
            * extension
        
        Usage example:
            file_obj = File("<file_name.txt>")
            file_obj.file_parts()
        
        Args:
            ext: File extension, needed if the file extension of file 
                object is longer than 4 characters.
        
        Returns:
            path: Absolute file path, excluding filename.
            filename: Filename, excluding extension.
            ext: File extension.
        '''
        
        file: str = self.file
        file = self.abs_path()
        
        if platform.system().lower() == "windows":
            [path, _filename] = os.path.splitdrive(file)
        else:
            [path, _filename] = os.path.split(file)
        
        if ext:
            ext_num = len(ext)
            _filename = _filename[:-(ext_num)]
            [filename, _ext] = os.path.splitext(_filename)
        elif self.ext:
            ext = self.ext
            ext_num = len(ext)
            _filename = _filename[:-(ext_num)]
            [filename, _ext] = os.path.splitext(_filename)
        else:
            [filename, ext] = os.path.splitext(_filename)
        
        return path, filename, ext

In [60]:
file_2 = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz"

In [61]:
t = "test.txt"

In [62]:
if t.count(".") is 1:
    print("go")

go


In [63]:
file_2.count(".")

4

In [64]:
f = File("test.txt")

In [65]:
f.file

'test.txt'

In [66]:
f.ext

'.txt'

In [67]:
f.rm_ext()

'test'

In [68]:
f.file_parts()

('/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck', 'test', '.txt')

In [69]:
f.file

'test.txt'

In [70]:
f.file_parts(".txt")

('/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck', 'test', '.txt')

In [71]:
f.abs_path()

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.txt'

In [28]:
os.path.split(f.abs_path())

('/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck', 'test.txt')

In [29]:
os.path.splitdrive(f.abs_path())

('', '/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.txt')

In [47]:
os.path.splitext(f.abs_path())

('/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test', '.txt')

In [31]:
os.path.split("test.txt")

('', 'test.txt')

In [34]:
f.file[-4:]

'.txt'

In [45]:
platform.system()

'Linux'

In [72]:
ff = File(file_2)

In [73]:
ff.file

'../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz'

In [74]:
ff.ext

'.nii.gz'

In [48]:
# os.path.split(ff.abs_path())[1]

In [49]:
# os.path.splitext(os.path.split(ff.abs_path())[1])

In [75]:
# ff.rm_ext(".nii.gz")
ff.rm_ext()

'../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi'

In [51]:
ff.file_parts()

('/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01',
 'sub-C01_ses-001_bval-b2000_run-01_dwi',
 '.nii.gz')

In [106]:
ff.file[:-7]

'../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi'

In [261]:
f.file_parts()

('/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck', 'test', '.txt')

In [15]:
f.write_txt()

'test.txt'

In [18]:
os.remove(f.file)

In [4]:
class TmpDir(object):
    '''Temporary directory class that creates temporary directories and files
    given a parent directory.
    
    Attributes (class and instance attributes):
        tmp_dir (class and instance): Temproary directory.
        parent_tmp_dir (class and instance): Input parent directory.
    '''
    
    # Set parent tmp directory, as tmp_dir is overwritten
    tmp_dir: str = ""
    parent_tmp_dir: str = ""
    
    def __init__(self,
                tmp_dir: str,
                use_cwd: bool = False) -> None:
        '''Init doc-string for TmpDir class.
        
        Usage example:
            tmp_directory = TmpDir("<temporary_directory>")
            tmp_directory.tmp_dir()   # Returns temporoary directory path str
        
        Args:
            tmp_dir: Temporary parent directory name/path.
            use_cwd: Use current working directory as working direcory.
        '''
        _n: int = 10000 # maximum N for random number generator
        tmp_dir: str = os.path.join(tmp_dir,'tmp_dir_' + 
                               str(random.randint(0,_n)))
        self.tmp_dir: str = tmp_dir
        self.parent_tmp_dir: str = os.path.dirname(self.tmp_dir)
        if use_cwd:
            _cwd = os.getcwd()
            self.tmp_dir = os.path.join(_cwd,self.tmp_dir)
            self.parent_tmp_dir = os.path.dirname(self.tmp_dir)
        
    def mk_tmp_dir(self) -> None:
        '''Creates/makes temporary directory.
        
        Usage example:
            tmp_directory = TmpDir("<temporary_directory>")
            tmp_directory.mk_tmp_dir()   # Makes temporary_directory
        '''
        if not os.path.exists(self.tmp_dir):
            return os.makedirs(self.tmp_dir)
        else:
            print("Temporary directory already exists")
        
    def rm_tmp_dir(self,
                  rm_parent: bool = False) -> None:
        '''Removes temporary directory.
        
        Usage example:
            tmp_directory = TmpDir("<temporary_directory>")
            tmp_directory.rm_tmp_dir()   # Removes temporary directory
        
        Args:
            rm_parent: Removes parent directory as well.
        '''
        if rm_parent and os.path.exists(self.parent_tmp_dir):
            return shutil.rmtree(self.parent_tmp_dir)
        elif os.path.exists(self.tmp_dir):
            return shutil.rmtree(self.tmp_dir)
        else:
            print("Temporary directory does not exist")
    
    class TmpFile(File):
        '''Child/sub-class of TmpDir. Creates temporary files by inheriting File object
        methods and properties from the File class.
        
        Attributes (class and instance attributes):
            tmp_file (class and instance): Temporary file name.
        '''
        
        tmp_file: File = ""
        tmp_dir: str = ""

        def __init__(self,
                    tmp_file: File,
                    tmp_dir: str = "") -> None:
            '''Init doc-string for TmpFile class. Allows for creation of 
            a temporary file in parent class' temporary directory location.
            TmpFile also inherits methods from the File class.
            
            Usage example:
                tmp_directory = TmpDir("<temporary_directory>")
                temp_file = TmpDir.TmpFile("<temporary_file>", tmp_directory.tmp_dir)
                temp_file.tmp_file   # Returns temporary filename as str
            
            Args:
                tmp_file: Temporary file name.
                tmp_dir: Temporary directory name.
            '''
            self.tmp_file = tmp_file
            self.tmp_dir = tmp_dir
            self.tmp_file = os.path.join(self.tmp_dir,self.tmp_file)
            File.__init__(self,self.tmp_file)

In [204]:
d = "test.dir"

In [205]:
dd = TmpDir(d, True)

In [206]:
dd.mk_tmp_dir()

In [207]:
f = 'test.file'

In [208]:
dd.tmp_dir

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.dir/tmp_dir_326'

In [209]:
ff = TmpDir.TmpFile(f,dd.tmp_dir)

In [210]:
ff.touch()

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.dir/tmp_dir_326/test.file'

In [211]:
ff.tmp_dir

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.dir/tmp_dir_326'

In [212]:
ff.abs_path()

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.dir/tmp_dir_326/test.file'

In [213]:
ff.tmp_file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.dir/tmp_dir_326/test.file'

In [214]:
ff.tmp_file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.dir/tmp_dir_326/test.file'

In [215]:
ff.touch()

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.dir/tmp_dir_326/test.file'

In [216]:
dd.rm_tmp_dir(True)

In [20]:
# os.remove("test.file")

In [5]:
class NiiFile(File):
    '''NIFTI file class object for handling NIFTI files. Inherits methods and 
    properties from the File class.
    
    Attributes (class and instance attributes):
        file (class and instance): NIFTI file path.
    '''
    def __init__(self,
                 file: File) -> None:
        '''Init doc-string for NiiFile class.
        
        Usage example:
            nii_file = NiiFile("<file.nii>")
            nii_file.file   # Returns NIFTI filename as str
        '''
        self.file = file
        File.__init__(self,self.file)


In [77]:
f = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz"

In [78]:
n_file = NiiFile(f)

In [79]:
n_file.file

'../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz'

In [80]:
n_file.ext

'.nii.gz'

In [251]:
# n_file.file = n_file.abs_path()
# n_file.file

In [81]:
n_file.rm_ext()

'../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi'

In [82]:
n_file.file_parts()

('/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01',
 'sub-C01_ses-001_bval-b2000_run-01_dwi',
 '.nii.gz')

In [6]:
class LogFile(File):
    '''Class that creates a log file for logging purposes. Due to how this 
    class is constructed - its intended use case requires that this class 
    is instantiated/called once and ONLY once.
    
    Once a class instance has been instantiated, then it and its associated
    methods can be used.
    
    Attributes (class and instance attributes):
        log_file: Log file filename.
    '''
    
    def __init__(self,
                 log_file: File = "") -> None:
        '''Init doc-string for LogFile class. Initiates logging and its 
        associated methods.
        
        Usage examples:
            log = LogFile("<file.log>")
            log.file   # Returns log file's filename
        
        Args:
            file: Log filename (need not exist at runtime).
        '''
        self.log_file: File = log_file
        
        # Set-up logging to file
        logging.basicConfig(level=logging.INFO,
                            format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                            datefmt='%d-%m-%y %H:%M:%S',
                            filename=self.log_file,
                            filemode='a')
        
        # Define a Handler which writes INFO messages or higher to the sys.stderr
        self.console = logging.StreamHandler()
        self.console.setLevel(logging.INFO)
        
        # Add the handler to the root logger
        logging.getLogger().addHandler(self.console)
        
        # Define logging
        self.logger = logging.getLogger(__name__)
        
    def info(self,
            msg: str = "") -> None:
        '''Writes information to log file.
        
        Usage examples:
            log = LogFile("<file.log>")
            log.info("<str>")
        
        Args:
            msg: String to be printed to log file.
        '''
        self.logger.info(msg)
        
    def debug(self,
            msg: str = "") -> None:
        '''Writes debug information to file.
        
        Usage examples:
            log = LogFile("<file.log>")
            log.debug("<str>")
        
        Args:
            msg: String to be printed to log file.
        '''
        self.logger.debug(msg)
        
    def error(self,
            msg: str = "") -> None:
        '''Writes error information to file.
        
        Usage examples:
            log = LogFile("<file.log>")
            log.error("<str>")
        
        Args:
            msg: String to be printed to log file.
        '''
        self.logger.error(msg)
        
    def warning(self,
            msg: str = "") -> None:
        '''Writes warnings to file.
        
        Usage examples:
            log = LogFile("<file.log>")
            log.warning("<str>")
        
        Args:
            msg: String to be printed to log file.
        '''
        self.logger.warning(msg)
    
    def log(self,
        log_cmd: str = ""):
        '''
        Log function for logging commands and messages to some log file.
        
        Usage examples:
            log = LogFile("<file.log>")
            log.log("<log_cmd>")
            
        Args:
            log_cmd: Message to be written to log file
        '''

        # Log command/message
        self.info(log_cmd)

In [34]:
# Needs to be instantiated once and ONLY once
# log = LogFile("sub-test.2.log")
log = LogFile(""); log.file

''

In [8]:
class Command(object):
    '''Creates a command and an empty command list for UNIX command line programs/applications. Primary use and
    use-cases are intended for the subprocess module and its associated classes (i.e. Popen/call/run).
    
    Attributes (class and instance attributes):
        command (instance): Command to be performed on the command line.
        cmd_list (instance): Mutable list that can be appended to.
    '''

    def __init__(self,
                 command: str) -> list:
        '''Init doc-string for Command class. Initializes a command to be used on UNIX command line.
        The input argument is a command (string), and a mutable list is returned (, that can later
        be appended to).
        
        Usage example:
            >>> echo = Command("echo")
            >>> echo.cmd_list.append("Hi!")
            >>> echo.cmd_list.append("I have arrived!")
        
        Arguments:
            command: Command to be used. Note: command used must be in system path
        
        Returns:
            cmd_list: Mutable list that can be appended to.
        '''
        self.command = command
        self.cmd_list = [f"{self.command}"]
        
    def check_dependency(self,
                         err_msg: Optional[str] = None) -> Union[bool,None]:
        '''Checks dependency of some command line executable. Should the 
        dependency not be met, then an exception is raised. Check the 
        system path should problems arise and ensure that the executable
        of interest is installed.
        
        Usage example:
            >>> figlet = Command("figlet")
            >>> figlet.check_dependency()   # Raises exception if not in system path
        
        Args:
            err_msg: Error message to print to screen.

        Returns:
            Returns True if dependency is met.
        
        Raises:
            DependencyError: Dependency error exception is raised if the dependency
                is not met.
        '''
        if not shutil.which(self.command):
            if err_msg:
                print(f"\n \t {err_msg} \n")
            else:
                print(f"\n \t The required dependency {self.command} is not installed or in the system path. \n")
            raise DependencyError("Command executable not found in system path.")
        else:
            return True
        
    def run(self,
            log: LogFile = "",
            debug: bool = False,
            dryrun: bool = False,
            env: dict = {},
            stdout: File = "",
            shell: bool = False
           ) -> Tuple[int,File,File]:
        '''Uses python's built-in subprocess class to execute (run) a command from an input command list.
        The standard output and error can optionally be written to file.
        
        NOTE: The contents of the 'stdout' output file will be empty if 'shell' is set to True.
        
        Usage example:
            >>> # Create command and cmd_list
            >>> echo = Command("echo")
            >>> echo.cmd_list.append("Hi!")
            >>> echo.cmd_list.append("I have arrived!")
            
            >>> # Run/execute command
            >>> echo.run()
            (0, '', '')
        
        Args:
            log: LogFile object
            debug: Sets logging function verbosity to DEBUG level
            dryrun: Dry run -- does not run task. Command is recorded to log file.
            env: Dictionary of environment variables to add to subshell.
            stdout: Output file to write standard output to.
            shell: Use shell to execute command.
            
        Returns:
            p.returncode: Return code for command execution should the 'log_file' option be used.
            stdout: Standard output writtent to file should the 'stdout' option be used.
            stderr: Standard error writtent to file should the 'stdout' option be used.
        '''
        
        # Create command str for log
        cmd: str = ' '.join(self.cmd_list) # Join list for logging purposes
        
        if log:
            if debug:
                log.debug(f"Running: {cmd}")
            else:
                log.info(f"Running: {cmd}")
        
        if log:
            if dryrun:
                log.info("Performing command as dryrun")
                return 0
            else:
                print("Performing command as dryrun")
                return 0
        
        # Define environment variables
        merged_env: dict = os.environ
        if env:
            merged_env.update(env)
        
        # Execute/run command
        p = subprocess.Popen(self.cmd_list,shell=shell,env=merged_env,
                        stdout=subprocess.PIPE,stderr=subprocess.PIPE)

        # Write log files
        out,err = p.communicate()
        out = out.decode('utf-8')
        err = err.decode('utf-8')

        # Write std output/error files
        if stdout:
            stderr: str = os.path.splitext(stdout)[0] + ".err"
                
            stdout: File = File(stdout)
            stderr: File = File(stderr)
            
            stdout.write_txt(out)
            stderr.write_txt(err)
        else:
            stdout = None
            stderr = None

        if p.returncode:
            if log:
                log.error(f"command: {cmd} \n Failed with returncode {p.returncode}")
            else:
                print(f"command: {cmd} \n Failed with returncode {p.returncode}")

        if len(out) > 0:
            if log:
                if debug:
                    log.debug(out)
                else:
                    log.info(out)

        if len(err) > 0:
            if log:
                if debug:
                    log.info(err)
                else:
                    log.warning(err)
            else:
                print(f"ERROR: {err}")
        return p.returncode,stdout,stderr

In [9]:
figlet = Command("figlet"); figlet.check_dependency()

True

str

In [10]:
fsl_math = Command("fslmaths")

In [11]:
fsl_math.cmd_list.append("-h")

In [12]:
fsl_math.cmd_list

['fslmaths', '-h']

In [13]:
fsl_math.run()

(0, None, None)

In [120]:
os.remove("sub-test.2.log")

In [39]:
fsl_math.check_dependency()

In [13]:
fsl_val = Command("fslval")

In [14]:
fsl_val.run()

Running: fslval
command: fslval 
 Failed with returncode 1
Usage: fslval <input> <keyword>



(1, None, None)